In [1]:
import pandas as pd

In [2]:
from torch import device, cuda

In [3]:
from sentence_transformers import SentenceTransformer, util

In [4]:
from tools import tools

In [5]:
device = device('cuda' if cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
model_name = 'all-mpnet-base-v2'

mpnet = SentenceTransformer(model_name)
mpnet.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [7]:
reports_file_path = 'data/br/firefox/firefox.csv'
relations_file_path = 'data/br/firefox/firefox_pairs.csv'

reports_firefox = pd.read_csv(reports_file_path, index_col='bug_id')
relations_firefox = pd.read_csv(relations_file_path)

In [8]:
reports_firefox_test = reports_firefox[0:1000]

In [9]:
# reports_firefox_test['mpnet'] = reports_firefox_test.apply(
#     lambda row: mpnet.encode(row['description']),
#     axis=1
# )

reports_firefox_test = tools.generate_encodings(
    reports_firefox_test,
    model_name
)

c:\Users\lpros\Documents\thesis\thesis-code\fine-tuning-experimentation\tools.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[model_name] = df.apply(


In [11]:
reports_firefox_test.head()

,priority,component,dup_id,short_desc,description,bug_status,resolution,version,creation_ts,delta_ts,all-mpnet-base-v2
bug_id,,,,,,,,,,,
10954,P3,Preferences,[],Dialup properties needs to be exposed in prefs,The dialup properties of the profile should be...,RESOLVED,WONTFIX,Trunk,1999-07-30 15:55:51 -0700,2008-05-14 11:44:15 -0700,"[-0.011274192, 0.09213826, -0.010780337, -0.03..."
14871,--,General,269442,[Find] Find whole word only,Please add Match Whole Word Only option to bro...,RESOLVED,DUPLICATE,Trunk,1999-09-24 14:49:34 -0700,2011-10-05 16:35:31 -0700,"[0.02882544, 0.049673, -0.0020014888, 0.043006..."
19118,--,Preferences,[],Plug-In Manager (ui for choosing mimetype-plug...,I would really like a plug-in manager for my b...,RESOLVED,WONTFIX,Trunk,1999-11-17 14:58:26 -0800,2013-01-29 11:48:39 -0800,"[0.0067006177, -0.0040191724, -0.031814296, 0...."
54746,P3,Preferences,[],Language encodings in font prefs dialog not so...,Language encodings are listed in a seemingly r...,RESOLVED,WORKSFORME,Trunk,2000-09-29 14:12:11 -0700,2013-02-27 15:47:29 -0800,"[0.013975114, -0.027381923, 0.012256341, 0.063..."
56892,P3,General,[],Synaptics touchpad scrolling not working,From Bugzilla Helper:; User-Agent: Mozilla/5.0...,RESOLVED,WORKSFORME,unspecified,2000-10-16 14:48:15 -0700,2009-10-14 11:38:29 -0700,"[0.002355588, 0.039769035, 0.0009729369, 0.016..."


In [11]:
i = 0
duplicate_ids = []
bug_id = 0
count = 0
for index, row in relations_firefox.iterrows():

    bug_id = row[0]

    if ( not pd.isna(row[1])):

        if (count == 7):
            duplicate_ids = [
                int(x) for x in row[1].split(";") if int(x) in reports_firefox_test.index
            ]

            duplicate_ids = set(duplicate_ids)
            break
        else:
            count+=1

print(bug_id)
    
        
duplicate_ids

110894


C:\Users\lpros\AppData\Local\Temp\ipykernel_14836\2216721754.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  bug_id = row[0]
C:\Users\lpros\AppData\Local\Temp\ipykernel_14836\2216721754.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if ( not pd.isna(row[1])):
C:\Users\lpros\AppData\Local\Temp\ipykernel_14836\2216721754.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  int(x) for x in row[1].split(";") if int(x) in report

{178756}

In [12]:
prompt = reports_firefox_test.loc[bug_id]
prompt
type(row)

pandas.core.series.Series

In [14]:
tools.rr_k(prompt=prompt, df=reports_firefox_test, model_name=model_name, duplicate_ids=duplicate_ids, k=10)

1.0